<img src='http://www-scf.usc.edu/~ghasemig/images/sharif.png' alt="SUT logo" width=500 height=300 align=center class="saturate" >

<br>
<font>
<div dir=ltr align=center>
<font color=0F5298 size=7>
    Artificial Intelligence <br>
<font color=2565AE size=5>
    Computer Engineering Department <br>
    Spring 2024<br>
<font color=3C99D size=5>
    Practical Assignment 3 - Heart Disease Prediction using Hidden Markov Models  <br>
<font color=696880 size=4>
Omid Daliran


# Personal Data

In [18]:
student_number = "400104801"
Name = "Mehran"
Last_Name = "Bakhtiari"

# Libraries

In [2]:
import csv
import numpy as np
import pandas as pd

# Q2: Heart Disease Prediction using Hidden Markov  (100 Points)

# Introduction


In this notebook, we explore the application of Hidden Markov Models (HMM) in predicting heart disease risk based on DNA sequences. Heart disease is a prevalent and life-threatening condition, and early detection is crucial for effective management. By leveraging HMM, we aim to identify regions within DNA sequences associated with high and low GC content, which have been linked to heart disease susceptibility.


# Hidden Markov Models (HMM) (40 points)

Hidden Markov Models are probabilistic models widely used in sequential data analysis, particularly in bioinformatics. They are characterized by states, transitions between states, and emission probabilities associated with each state. In our context, the states represent the underlying biological characteristics of DNA sequences, while the emission probabilities signify the likelihood of observing specific nucleotides given each state.

To facilitate our heart disease classification, we'll implement a custom Hidden Markov Model (HMM) class in Python. This class will include the Viterbi and Forward algorithms.

In [3]:
class HMM:
    def __init__(self, states: list, emissions: list, start_probabilities: list, transition_probabilities: list):
        self.states = states
        self.state_labels = {i: state for i, state in enumerate(states)}
        self.emissions = emissions
        self.start_probabilities = start_probabilities
        self.transition_probabilities = transition_probabilities

    def viterbi(self, sequence: str) -> tuple:
        T = len(sequence)
        N = len(self.states)
        viterbi_matrix = [[0] * N for _ in range(T)]
        backtrack_matrix = [[0] * N for _ in range(T)]

        for i, state in enumerate(self.states):
            viterbi_matrix[0][i] = self.start_probabilities[i] * self.emissions[i][sequence[0]]

        for t in range(1, T):
            for j in range(N):
                max_prob = 0
                max_state = 0
                for i in range(N):
                    prob = viterbi_matrix[t-1][i] * self.transition_probabilities[i][j]
                    if prob > max_prob:
                        max_prob = prob
                        max_state = i
                viterbi_matrix[t][j] = max_prob * self.emissions[j][sequence[t]]
                backtrack_matrix[t][j] = max_state

        path = [0] * T
        max_probability = max(viterbi_matrix[-1])
        path[-1] = viterbi_matrix[-1].index(max_probability)
        for t in range(T - 2, -1, -1):
            path[t] = backtrack_matrix[t + 1][path[t + 1]]

        return [self.states[i] for i in path], max_probability

    def forward_algorithm(self, sequence: str) -> list:
        T = len(sequence)
        N = len(self.states)
        forward_matrix = [[0] * N for _ in range(T)]

        for i, state in enumerate(self.states):
            forward_matrix[0][i] = self.start_probabilities[i] * self.emissions[i][sequence[0]]

        for t in range(1, T):
            for j in range(N):
                prob = sum(
                    forward_matrix[t - 1][i] * self.transition_probabilities[i][j] * self.emissions[j][sequence[t]] for
                    i in range(N))
                forward_matrix[t][j] = prob

        probabilities_last_states = [forward_matrix[-1][i] for i in range(N)]

        total_probability = sum(probabilities_last_states)
        probabilities_last_states = [prob / total_probability for prob in probabilities_last_states]

        return probabilities_last_states

GC content, the proportion of guanine (G) and cytosine (C) nucleotides in a DNA sequence, is a fundamental genomic feature. Variations in GC content have been implicated in various biological processes, including gene regulation and disease susceptibility. Specifically, high GC content (H) regions are associated with increased gene expression and potential regulatory elements, while low GC content (L) regions may indicate structural elements or regions prone to mutations. The transition and emission and start probabilities are as shown in the image below:

![Probabilities](Probabilities.png)

In [4]:
states = ['H', 'L']
emissions = [
    {'A': 0.20, 'C': 0.30, 'G': 0.30, 'T': 0.20},
    {'A': 0.30, 'C': 0.20, 'G': 0.20, 'T': 0.30}
]

start_probabilities = [0.5, 0.5]

transition_probabilities = [
    [0.5, 0.5],  
    [0.4, 0.6]  
]

hmm = HMM(states, emissions, start_probabilities, transition_probabilities)

sequence = "ATGCGCGATCGATCGAATCGCGT"

path, max_probability = hmm.viterbi(sequence)
print("Viterbi Path:", path)
print("Max Probability (Viterbi):", max_probability)

probability = hmm.forward_algorithm(sequence)
print("Probability (Forward Algorithm):", probability)

Viterbi Path: ['L', 'L', 'H', 'H', 'H', 'H', 'H', 'L', 'L', 'H', 'H', 'L', 'L', 'H', 'H', 'L', 'L', 'L', 'H', 'H', 'H', 'H', 'L']
Max Probability (Viterbi): 1.1438396227480495e-19
Probability (Forward Algorithm): [0.3581800885349868, 0.6418199114650132]


# Calculating Probabilities for Blood Type, Height, and Weight (10 points)

After obtaining the sequence of H and L regions from the DNA using the Viterbi algorithm, we proceed to calculate the probabilities for other attributes such as blood type, height, weight, and the presence or absence of DCM (DCM, or Dilated Cardiomyopathy, is a heart condition characterized by the enlargement of the heart's left ventricle. It can lead to heart failure if not properly managed.) using the Forward algorithm. In this step, we aim to leverage the Hidden Markov Model (HMM) to infer these attributes based on the observed GC content sequence.


We will use the functionality of the Forward algorithm to compute the probabilities associated with each possible state after observing the sequence.

Attributes and Possible States:
* Blood Type: O, A, B, AB
* Height: 160 cm, 170 cm, 180 cm, 190 cm
* Weight: 50 kg, 65 kg, 80 kg, 95 kg
* DCM: Y (indicating presence) or N (indicating absence)


In [5]:
def probabilities_from_sequence(sequence: str) -> tuple:
    height_emissions = [{'H': 0.2, 'L': 0.8}, {'H': 0.3, 'L': 0.7},
                        {'H': 0.9, 'L': 0.1}, {'H': 0.8, 'L': 0.2}]

    weight_emissions = [{'H': 0.3, 'L': 0.7}, {'H': 0.1, 'L': 0.9},
                        {'H': 0.2, 'L': 0.8}, {'H': 0.7, 'L': 0.3}]

    blood_type_emissions = [{'H': 0.75, 'L': 0.25}, {'H': 0.8, 'L': 0.2},
                            {'H': 0.85, 'L': 0.15}, {'H': 0.7, 'L': 0.3}]

    DCM_emissions = [{'H': 0.7, 'L': 0.3}, {'H': 0.6, 'L': 0.4}]

    height_transition_probabilities = [[0.7, 0.2, 0.05, 0.05], [0.1, 0.8, 0.05, 0.05], [0.1, 0.1, 0.7, 0.1], [0.1, 0.1, 0.1, 0.7]]
    weight_transition_probabilities = [[0.7, 0.2, 0.05, 0.05], [0.1, 0.8, 0.05, 0.05], [0.1, 0.1, 0.7, 0.1], [0.1, 0.1, 0.1, 0.7]]
    blood_type_transition_probabilities = [[0.7, 0.2, 0.05, 0.05], [0.1, 0.8, 0.05, 0.05], [0.1, 0.1, 0.7, 0.1], [0.1, 0.1, 0.1, 0.7]]
    DCM_transition_probabilities = [[0.9, 0.1], [0.1, 0.9]]

    height_start_probabilities = [0.1, 0.4, 0.4, 0.1]
    weight_start_probabilities = [0.15, 0.35, 0.35, 0.15]
    blood_type_start_probabilities = [0.1, 0.35, 0.35, 0.2]
    DCM_start_probabilities = [0.25, 0.75]

    height_hmm = HMM(['160 cm', '170 cm', '180 cm', '190 cm'], height_emissions, height_start_probabilities, height_transition_probabilities)
    weight_hmm = HMM(['50 kg', '65 kg', '80 kg', '95 kg'], weight_emissions, weight_start_probabilities, weight_transition_probabilities)
    blood_type_hmm = HMM(['O', 'A', 'B', 'AB'], blood_type_emissions, blood_type_start_probabilities, blood_type_transition_probabilities)
    DCM_hmm = HMM(['Y', 'N'], DCM_emissions, DCM_start_probabilities, DCM_transition_probabilities)

    height_probabilities = height_hmm.forward_algorithm(sequence)
    weight_probabilities = weight_hmm.forward_algorithm(sequence)
    blood_type_probabilities = blood_type_hmm.forward_algorithm(sequence)
    DCM_probabilities = DCM_hmm.forward_algorithm(sequence)

    total_height_probability = sum(height_probabilities)
    height_probability_dic = {state: prob / total_height_probability for state, prob in zip(height_hmm.states, height_probabilities)}

    total_weight_probability = sum(weight_probabilities)
    weight_probability_dic = {state: prob / total_weight_probability for state, prob in zip(weight_hmm.states, weight_probabilities)}

    total_blood_type_probability = sum(blood_type_probabilities)
    blood_type_probability_dic = {state: prob / total_blood_type_probability for state, prob in zip(blood_type_hmm.states, blood_type_probabilities)}

    total_DCM_probability = sum(DCM_probabilities)
    DCM_probability_dic = {state: prob / total_DCM_probability for state, prob in zip(DCM_hmm.states, DCM_probabilities)}

    return height_probability_dic, weight_probability_dic, blood_type_probability_dic, DCM_probability_dic


In [6]:
sequence = "HHLLHHHLLHLLLLHHLL"

height_probability, weight_probability, blood_type_probability, DCM_probability = probabilities_from_sequence(sequence)

print("Probability of height given sequence:", height_probability)
print("Probability of weight given sequence:", weight_probability)
print("Probability of blood type given sequence:", blood_type_probability)
print("Probability of DCM given sequence:", DCM_probability)

Probability of height given sequence: {'160 cm': 0.3660593174871113, '170 cm': 0.5760951509787055, '180 cm': 0.01585035336762777, '190 cm': 0.04199517816655548}
Probability of weight given sequence: {'50 kg': 0.28459529088928676, '65 kg': 0.4270707539024914, '80 kg': 0.2095735632296391, '95 kg': 0.07876039197858277}
Probability of blood type given sequence: {'O': 0.2962828473309036, 'A': 0.33050805731904764, 'B': 0.08789883086305705, 'AB': 0.2853102644869917}
Probability of DCM given sequence: {'Y': 0.3297758181008331, 'N': 0.6702241818991669}


# Bayesian Network for Heart Disease Probability Calculation (30 points)

In this section, we introduce a Bayesian network to calculate the probability of heart disease based on various features. We assume that the relationships between features follow a specific structure, depicted in the image below.

![Bayesian Net](BN.png)

The Bayesian network comprises nodes representing different features, including sex, weight, height, Dilated Cardiomyopathy (DCM), blood type, and heart disease. The directed edges between nodes indicate probabilistic dependencies between them.

Function Implementation:
We will implement two functions to facilitate the calculation of heart disease probability:

1. calculate_sex_probability:

  This function calculates the probability of sex based on the given probabilities of weight and height.

2. calculate_heart_disease_probability:

  This function computes the probability of heart disease based on the probabilities of sex, DCM, and blood type.

In [7]:
P_sex_given_hw = {
    ('M', 160, 50): 0.7, ('F', 160, 50): 0.3,
    ('M', 160, 65): 0.2, ('F', 160, 65): 0.8,
    ('M', 160, 80): 0.3, ('F', 160, 80): 0.7,
    ('M', 160, 95): 0.2, ('F', 160, 95): 0.8,
    ('M', 170, 50): 0.8, ('F', 170, 50): 0.2,
    ('M', 170, 65): 0.75, ('F', 170, 65): 0.25,
    ('M', 170, 80): 0.4, ('F', 170, 80): 0.6,
    ('M', 170, 95): 0.3, ('F', 170, 95): 0.7,
    ('M', 180, 50): 0.9, ('F', 180, 50): 0.1,
    ('M', 180, 65): 0.7, ('F', 180, 65): 0.3,
    ('M', 180, 80): 0.65, ('F', 180, 80): 0.35,
    ('M', 180, 95): 0.4, ('F', 180, 95): 0.6,
    ('M', 190, 50): 0.95, ('F', 190, 50): 0.05,
    ('M', 190, 65): 0.8, ('F', 190, 65): 0.2,
    ('M', 190, 80): 0.6, ('F', 190, 80): 0.4,
    ('M', 190, 95): 0.95, ('F', 190, 95): 0.05
}

In [8]:
def calculate_sex_probability(height_prob: dict, weight_prob: dict) -> dict:
    sex_dic = {'M': 0, 'F': 1}

    sex_probability = np.zeros(2)

    for sex, height, weight in P_sex_given_hw.keys():
        sex_probability[sex_dic[sex]] += P_sex_given_hw[(sex, height, weight)] * height_prob[height] * weight_prob[weight]

    return {'M': sex_probability[0], 'F': sex_probability[1]}

In [9]:
P_height = {160: 0.1, 170: 0.3, 180: 0.4, 190: 0.2}
P_weight = {50: 0.2, 65: 0.3, 80: 0.4, 95: 0.1}

sex_probabilities = calculate_sex_probability(P_height, P_weight)
print("Probability of sex:", sex_probabilities)

Probability of sex: {'M': 0.6355000000000002, 'F': 0.36450000000000005}


In [10]:
P_heart_disease_given_parents = {
    ('N', 'M', 'O', 'Y'): 0.3, ('Y', 'M', 'O', 'Y'): 0.7,
    ('N', 'M', 'A', 'Y'): 0.4, ('Y', 'M', 'A', 'Y'): 0.6,
    ('N', 'M', 'B', 'Y'): 0.5, ('Y', 'M', 'B', 'Y'): 0.5,
    ('N', 'M', 'AB', 'Y'): 0.6, ('Y', 'M', 'AB', 'Y'): 0.4,
    ('N', 'F', 'O', 'Y'): 0.6, ('Y', 'F', 'O', 'Y'): 0.4,
    ('N', 'F', 'A', 'Y'): 0.3, ('Y', 'F', 'A', 'Y'): 0.7,
    ('N', 'F', 'B', 'Y'): 0.4, ('Y', 'F', 'B', 'Y'): 0.6,
    ('N', 'F', 'AB', 'Y'): 0.7, ('Y', 'F', 'AB', 'Y'): 0.3,
    ('N', 'M', 'O', 'N'): 0.6, ('Y', 'M', 'O', 'N'): 0.4,
    ('N', 'M', 'A', 'N'): 0.15, ('Y', 'M', 'A', 'N'): 0.85,
    ('N', 'M', 'B', 'N'): 0.4, ('Y', 'M', 'B', 'N'): 0.6,
    ('N', 'M', 'AB', 'N'): 0.7, ('Y', 'M', 'AB', 'N'): 0.3,
    ('N', 'F', 'O', 'N'): 0.8, ('Y', 'F', 'O', 'N'): 0.2,
    ('N', 'F', 'A', 'N'): 0.7, ('Y', 'F', 'A', 'N'): 0.3,
    ('N', 'F', 'B', 'N'): 0.65, ('Y', 'F', 'B', 'N'): 0.35,
    ('N', 'F', 'AB', 'N'): 0.35, ('Y', 'F', 'AB', 'N'): 0.65,
}

In [11]:
def calculate_heart_disease_probability(DCM_prob: dict, sex_prob: dict, blood_type_prob: dict) -> dict:
    heart_disease_dic = {'Y': 0, 'N': 1}

    heart_disease_probability = np.zeros(2)

    for heart_disease, sex, blood_type, DCM in P_heart_disease_given_parents.keys():
        heart_disease_probability[heart_disease_dic[heart_disease]] += P_heart_disease_given_parents[(heart_disease, sex, blood_type, DCM)] * DCM_prob[DCM] * sex_prob[sex] * blood_type_prob[blood_type]

    return {'Y': heart_disease_probability[0], 'N': heart_disease_probability[1]}

In [12]:
P_DCM = {'Y': 0.2, 'N': 0.8}
P_blood_type = {'O': 0.4, 'A': 0.3, 'B': 0.2, 'AB': 0.1}
P_sex = {'M': 0.3, 'F': 0.7}

heart_disease_probabilities = calculate_heart_disease_probability(P_DCM, P_sex, P_blood_type)
print("Probability of heart disease: ",heart_disease_probabilities)

Probability of heart disease:  {'Y': 0.4152, 'N': 0.5848}


# Loading DNA Sequences and Heart Disease Probability Calculation (20 points)

In this part, we will load the DNA sequences from the file DNA_patients.csv and calculate the probability of heart disease for each patient. Subsequently, we will classify the patients based on their heart disease probability and save the data.

In [14]:
df = pd.read_csv('DNA_patients.csv')

df.head()

,ID,DNA
0,1,CTCCAATACCCCCCACAAGAACACACCCATAAAATTGCAACCCACA...
1,2,TTGATGTAGAAGTATATTTGTTGGGTATTTGAGGTAACGTTATTAG...
2,3,ATATAATTTAAAGTCACTGGAAAAAAACAACCTAATAAAAACCACC...
3,4,TAAAGACAAAATTAAATTGAAGTAATGTTATGTTAAAATTTTGAAT...
4,5,AGCGTTTGTTCGTTAGCCGTAGGCAATGACGTGATTCAGGTCTGTG...


In [15]:
DNAs = df['DNA']

states = ['H', 'L']
emissions = [
    {'A': 0.20, 'C': 0.30, 'G': 0.30, 'T': 0.20},
    {'A': 0.30, 'C': 0.20, 'G': 0.20, 'T': 0.30}
]

start_probabilities = [0.5, 0.5]

transition_probabilities = [
    [0.5, 0.5],  
    [0.4, 0.6]  
]

hmm = HMM(states, emissions, start_probabilities, transition_probabilities)

GC_content_seqs = []

for seq in DNAs:
  path, max_probability = hmm.viterbi(seq)
  GC_content_seqs.append(path)

In [16]:
heart_disease_classified = []

for seq in GC_content_seqs:
    height_probability_dic, weight_probability_dic, blood_type_probability_dic, DCM_probability_dic = probabilities_from_sequence(seq)
    height_probability_dic_int = {int(key.split()[0]): value for key, value in height_probability_dic.items()}
    weight_probability_dic_int = {int(key.split()[0]): value for key, value in weight_probability_dic.items()}
    sex_probability_dic = calculate_sex_probability(height_probability_dic_int, weight_probability_dic_int)
    heart_disease_probability_dic = calculate_heart_disease_probability(DCM_probability_dic, sex_probability_dic, blood_type_probability_dic)
    if heart_disease_probability_dic['Y'] > 0.5:
        heart_disease_classified.append(1)
    else:
        heart_disease_classified.append(0)

In [17]:
labels = [(i+1, heart_disease_classified[i]) for i in range(len(heart_disease_classified))]
csv_filename = "heart_disease_result.csv"
with open(csv_filename, mode='w', newline='') as file:
    writer = csv.writer(file)
    writer.writerow(['ID', 'Label'])
    for id, label in labels:
        writer.writerow([id, label])

**Note: You should upload heart_disease_result.csv along side your notebook.**